In [2]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler

SEED = 2017

Using TensorFlow backend.


### Load data

In [4]:
data = pd.read_csv('data/winequality-red.csv', sep=';')
y = data['quality']
X = data.drop(['quality'], axis=1)

### Split data into train and test sets

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [6]:
print('Average quality training set: {:.4f}'.format(y_train.mean()))
X_train.head()

Average quality training set: 5.6231


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
1140,7.3,0.40,0.30,1.7,0.080,33.0,79.0,0.99690,3.41,0.65,9.5
920,9.6,0.41,0.37,2.3,0.091,10.0,23.0,0.99786,3.24,0.56,10.5
1198,7.7,0.26,0.26,2.0,0.052,19.0,77.0,0.99510,3.15,0.79,10.9
423,10.5,0.24,0.47,2.1,0.066,6.0,24.0,0.99780,3.15,0.90,11.0
601,13.2,0.46,0.52,2.2,0.071,12.0,35.0,1.00060,3.10,0.56,9.0


### Normalize the input data

In [7]:
scaler = StandardScaler().fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))


### Determine baseline predictions

In [11]:
# Predict the mean quality of the training data for each validation input
print('MSE:', np.mean((y_test - ([y_train.mean()] * y_test.shape[0])) ** 2))

MSE: 0.5939855630834515


### Build our neural network by defining the network architecture

In [12]:
model = Sequential()
# First hidden layer with 100 hidden units
model.add(Dense(200, input_dim=X_train.shape[1], activation='relu')) 
# Second hidden layer with 50 hidden units
model.add(Dense(25, activation='relu'))
# Output layer
model.add(Dense(1, activation='linear'))
# Set optimizer
opt = Adam()
# Compile model
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

### Define the callback for early stopping and saving the best model

In [17]:
callbacks = [
             EarlyStopping(monitor='val_acc', patience=20, verbose=2),
             ModelCheckpoint('data/multi_layer_best_model.h5', monitor='val_acc', save_best_only=True, verbose=0)
            ]

### Run the model with a batch size of 64, 5000 epochs, and a validation split of 20 percent

In [18]:
batch_size = 64
n_epochs = 5000

In [19]:
model.fit(X_train.values, y_train, batch_size=64, epochs=n_epochs, validation_split=0.2, verbose=2, validation_data=(X_test.values, y_test), callbacks=callbacks)

Train on 1279 samples, validate on 320 samples
Epoch 1/5000
 - 0s - loss: 3.1660 - acc: 0.2275 - val_loss: 2.7110 - val_acc: 0.2625
Epoch 2/5000
 - 0s - loss: 2.3093 - acc: 0.2713 - val_loss: 2.4078 - val_acc: 0.2406
Epoch 3/5000
 - 0s - loss: 1.9772 - acc: 0.2909 - val_loss: 2.0660 - val_acc: 0.2812
Epoch 4/5000
 - 0s - loss: 1.7738 - acc: 0.2932 - val_loss: 1.8961 - val_acc: 0.2750
Epoch 5/5000
 - 0s - loss: 1.6214 - acc: 0.3041 - val_loss: 1.7532 - val_acc: 0.2844
Epoch 6/5000
 - 0s - loss: 1.4854 - acc: 0.3174 - val_loss: 1.6350 - val_acc: 0.3063
Epoch 7/5000
 - 0s - loss: 1.3710 - acc: 0.3276 - val_loss: 1.5448 - val_acc: 0.3156
Epoch 8/5000
 - 0s - loss: 1.2681 - acc: 0.3385 - val_loss: 1.4380 - val_acc: 0.3469
Epoch 9/5000
 - 0s - loss: 1.1728 - acc: 0.3495 - val_loss: 1.3554 - val_acc: 0.3625
Epoch 10/5000
 - 0s - loss: 1.0830 - acc: 0.3675 - val_loss: 1.2725 - val_acc: 0.3594
Epoch 11/5000
 - 0s - loss: 1.0032 - acc: 0.3855 - val_loss: 1.1985 - val_acc: 0.3719
Epoch 12/5000
 -

### print the performance on the test set after loading the optimal weights

In [21]:
best_model = model
best_model.load_weights('data/multi_layer_best_model.h5')
best_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Evaluate on test set
score = best_model.evaluate(X_test.values, y_test, verbose=0)
print('Test accuracy: %.2f%%' % (score[1]*100))

# Test accuracy: 65.62% 
# Benchmark accuracy on dataset 62.4%

Test accuracy: 63.44%
